In [1]:
import system_of_cities as sc
import linked_census as lc
import network_utils as nu
import pandas as pd
import numpy as np
from Plotting.Map import system_of_cities_map
from Plotting.Zipf import zipf_exponents_barchart, zipf_plots
from Plotting.CommunityQuality import clustered_adjacency_matrix_heatmap, community_block_matrix_heatmap
from Plotting.Layout import MapPlotLayout
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [19]:
years = [lc.CensusYear.y1860,lc.CensusYear.y1870]

In [20]:
collections = []
for y in tqdm(years):
    parent = sc.get_system_of_cities(year=y,backbone_method=nu.BackBoneMethod.NO)
    collection = parent.identify_subsystems_via_community_detection(algorithm=nu.CommunityDetectionAlgorithm.LEIDEN, resolution=0.001, n_iterations=10)
    collections.append(collection)

  0%|          | 0/2 [00:00<?, ?it/s]2023-09-18 13:39:15,610 - system_of_cities - DEBUG - get_system_of_cities: called with year 1860, sid 0, number of cities None, industry ALL, place_cluster_level 5, backbone_method no_filter, and kwargs {}
2023-09-18 13:39:15,611 - system_of_cities - DEBUG - get_system_of_cities: loading migration data
2023-09-18 13:39:15,611 - system_of_cities - DEBUG - get_system_of_cities: filtering migration data
2023-09-18 13:39:20,297 - system_of_cities - DEBUG - get_system_of_cities: loading city data
2023-09-18 13:39:20,312 - system_of_cities - DEBUG - get_system_of_cities: building city network
2023-09-18 13:39:20,313 - system_of_cities - DEBUG - _get_city_network: called
2023-09-18 13:39:20,319 - system_of_cities - DEBUG - _get_city_network: building city network
2023-09-18 13:39:25,445 - system_of_cities - DEBUG - _get_city_network: built network with 10630 vertices and 303213 edges
2023-09-18 13:39:25,447 - system_of_cities - DEBUG - _get_city_network: g

In [21]:
min_n_cities = 20
cutoff_thresholds = [100, 1000, 2000]

In [22]:
for c in collections:
    fig = system_of_cities_map(collection=c, min_n_cities=min_n_cities, layout=MapPlotLayout())
    fig.update_layout(width=1000, height=600)
    fig.write_image(f"/Users/anmusso/Desktop/Temp/Map_{c.year.value}_r={0.001}.png")
    fig = community_block_matrix_heatmap(collection=c, min_n_cities=min_n_cities, log_scale=True)
    fig.update_layout(width=1000, height=1000)
    fig.write_image(f"/Users/anmusso/Desktop/Temp/Block_{c.year.value}_r={0.001}.png")

In [13]:
fig = community_block_matrix_heatmap(collection=collections[0], min_n_cities=min_n_cities, log_scale=True)
fig.show(renderer="browser")

[[ 0.          0.08319744  0.08124264  0.09143777  0.09549745  0.09801125
   0.10286518  0.09044119  0.10909535  0.11402913  0.15962196  0.17133767
   0.19892461  0.16897857  0.21138845  0.21698291  0.27707828  0.21866937
   0.17799994  0.23032492  0.23536615  0.24014069]
 [ 0.08319744  0.          0.08401103  0.10714958  0.09677281  0.10141228
   0.08930504  0.11212849  0.10281211  0.10194434  0.16592517  0.19912667
   0.12572171  0.1316507   0.26382789  0.28978211  0.33072216  0.4222796
   0.25696292  0.26943389  0.38513703  0.35724353]
 [ 0.08124264  0.08401103  0.          0.11826557  0.1114735   0.11416574
   0.11053189  0.12286112  0.13603822  0.13245571  0.21366419  0.2240149
   0.17688819  0.17779298  0.30266341  0.28385799  0.36170099  0.42386362
   0.28478393  0.38513703  1.24266987  0.65691748]
 [ 0.09143777  0.10714958  0.11826557  0.          0.09290441  0.14826053
   0.13673489  0.09615802  0.20248664  0.18309893  0.15263912  0.16502582
   0.34434417  0.26214072  0.189639

In [16]:
fig = zipf_exponents_barchart(collection=collections[0], cutoff_thresholds=cutoff_thresholds, min_n_cities=min_n_cities, r2_threshold=0)
fig.show(renderer="browser")

In [23]:
for c in collections:
    figs = zipf_plots(collection=c, cutoff_threshold=1000, min_n_cities=min_n_cities)
    for i, fig in enumerate(figs):
        fig.update_layout(width=1000, height=600)
        fig.write_image(f"/Users/anmusso/Desktop/Temp/Zipf_{c.year.value}_r={0.001}_{i + 1}.png")

In [47]:
fig = community_block_matrix_heatmap(collection=collections[0], min_n_cities=min_n_cities, log_scale=True)
fig.show(renderer="browser")

(59, 17)


ValueError: The matrix argument must be square.

In [38]:
fig = community_block_matrix_heatmap(collection=c, n=n)
fig.show(renderer="browser")

In [31]:
import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, leaves_list

def order_distance_matrix(matrix):
    # Ensure the matrix is square
    assert matrix.shape[0] == matrix.shape[1], "Matrix must be square"
    
    # Convert the distance matrix into a condensed form
    condensed_matrix = squareform(matrix)
    
    # Compute the hierarchical clustering
    linked = linkage(condensed_matrix, method='single')
    
    # Get the order of rows (or columns) from the hierarchy
    order = leaves_list(linked)
    
    # Reorder the matrix
    ordered_matrix = matrix[order, :]
    ordered_matrix = ordered_matrix[:, order]
    
    return ordered_matrix